# First week submission

## In this notebook you can Only find How to group  data for making a baseline model and submit prediction for checking.
## In the next notebooks I will make an EDA(Exploratory Data Analysis) and make models from KNN to XGBoost

In [1]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
TrainSet = pd.read_csv(os.path.join('Data/sales_train.csv.gz'))

TestSet = pd.read_csv(os.path.join('Data/test.csv.gz'))

Items = pd.read_csv(os.path.join('Data/items.csv'))

ItemCategories = pd.read_csv(os.path.join('Data/item_categories.csv'))

Shops = pd.read_csv(os.path.join('Data/shops.csv'))

Submission = pd.read_csv(os.path.join('Data/sample_submission.csv.gz'))

In [3]:
print(TrainSet.shape)

print(TestSet.shape)

(2935849, 6)
(214200, 3)


In [4]:
TestSet

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [5]:
def CheckNaNValues(DataSet, NameOfDataSet):
    
    print("\nNaN values in ", NameOfDataSet, ": ", DataSet.isnull().sum().max())

CheckNaNValues(TrainSet, 'Train')

CheckNaNValues(TestSet, 'Test')

CheckNaNValues(Items, 'Items')

CheckNaNValues(ItemCategories, 'ItemCategories')

CheckNaNValues(Shops, 'Shops')


NaN values in  Train :  0

NaN values in  Test :  0

NaN values in  Items :  0

NaN values in  ItemCategories :  0

NaN values in  Shops :  0


In [6]:
def RemakesDateDataFunction(DataSet, NameOfColumn):
    
    DataSet[NameOfColumn] = pd.to_datetime(TrainSet[NameOfColumn], format = '%d.%m.%Y')

    DataSet.sort_values(by = [NameOfColumn], inplace = True)

    DataSet['Day'] = DataSet[NameOfColumn].dt.day

    DataSet['WeekDay'] = DataSet[NameOfColumn].dt.weekday

    DataSet['Month'] = DataSet[NameOfColumn].dt.month
    
    DataSet['Year'] = DataSet[NameOfColumn].dt.year
    
    return DataSet

TrainSet = RemakesDateDataFunction(TrainSet, 'date')

In [7]:
TrainSet

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,Day,WeekDay,Month,Year
49800,2013-01-01,0,18,5823,2500.0,1.0,1,1,1,2013
29784,2013-01-01,0,27,5573,849.0,1.0,1,1,1,2013
35476,2013-01-01,0,7,1006,399.0,1.0,1,1,1,2013
8330,2013-01-01,0,19,17707,899.0,1.0,1,1,1,2013
57384,2013-01-01,0,14,19548,149.0,1.0,1,1,1,2013
...,...,...,...,...,...,...,...,...,...,...
2885098,2015-10-31,33,41,21386,169.0,1.0,31,5,10,2015
2930981,2015-10-31,33,21,988,199.0,1.0,31,5,10,2015
2885097,2015-10-31,33,41,21377,169.0,1.0,31,5,10,2015
2930993,2015-10-31,33,22,10207,1199.0,1.0,31,5,10,2015


In [8]:
Submission

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


In [9]:
ItemsInMonth = TrainSet.groupby(['shop_id', 'item_id', 'Month'])['item_cnt_day'].sum()

Shops = TrainSet.groupby(['shop_id', 'item_id', 'Month'])['item_cnt_day'].sum().index.get_level_values(0)

ItemsID = TrainSet.groupby(['shop_id', 'item_id', 'Month'])['item_cnt_day'].sum().index.get_level_values(1)

In [10]:
ItemsInMonth = np.array(ItemsInMonth)

ItemsInMonth = pd.DataFrame(ItemsInMonth)

ItemsInMonth.insert(loc = 0, column = 'shop_id', value = Shops)

ItemsInMonth.insert(loc = 1, column = 'item_id', value = ItemsID)

ItemsInMonth.rename(columns = {0 : 'item_cnt_month'}, inplace = True)

ItemsInMonth

,shop_id,item_id,item_cnt_month
0,0,30,31.0
1,0,31,11.0
2,0,32,6.0
3,0,32,10.0
4,0,33,3.0
...,...,...,...
1396966,59,22164,2.0
1396967,59,22164,1.0
1396968,59,22167,1.0
1396969,59,22167,1.0


In [45]:
def CountAndShowUniqueValuesFunction(DataSet, NameOfDataSet, NameOfColumn, Normalization):
    
    print("For ", NameOfDataSet, ", column: ", NameOfColumn)

    if DataSet[NameOfColumn].value_counts().count() < 15:

        print("Number of uniques: ", DataSet[NameOfColumn].value_counts().count())
        
        print("Unique Values: \n", DataSet[NameOfColumn].value_counts(normalize = Normalization))
    
    else:
        
        print("Number of uniques: ", DataSet[NameOfColumn].value_counts().count(), "\n")

In [46]:
CountAndShowUniqueValuesFunction(ItemsInMonth, 'TrainSet', 'shop_id', True)

CountAndShowUniqueValuesFunction(ItemsInMonth, 'TrainSet', 'item_id', True)

CountAndShowUniqueValuesFunction(TestSet, 'TestSet', 'shop_id', True)

CountAndShowUniqueValuesFunction(TestSet, 'TestSet', 'item_id', True)

For  TrainSet , column:  shop_id
Number of uniques:  60 

For  TrainSet , column:  item_id
Number of uniques:  21807 

For  TestSet , column:  shop_id
Number of uniques:  42 

For  TestSet , column:  item_id
Number of uniques:  5100 



In [73]:
ItemsInMonth = ItemsInMonth.sort_values(by = 'shop_id')

TestSet = TestSet.sort_values(by = 'shop_id')

print(set(ItemsInMonth['shop_id'].unique()) - set(TestSet['shop_id'].unique()))

print(set(TestSet['shop_id'].unique()) - set(ItemsInMonth['shop_id'].unique()))

DifferenceBetweenUniqueValues = set(ItemsInMonth['shop_id'].unique()) - set(TestSet['shop_id'].unique())

{0, 1, 32, 33, 8, 9, 40, 11, 43, 13, 17, 51, 20, 54, 23, 27, 29, 30}
set()


In [ ]:
X = ItemsInMonth.drop(['item_cnt_month'], axis = 1)

Y = ItemsInMonth['item_cnt_month']

TestSet = TestSet.drop(['ID'], axis = 1)

Submission = Submission.drop(['item_cnt_month'], axis = 1)

In [ ]:
from xgboost import XGBRegressor

Model = XGBRegressor(n_estimators = 1000, max_depth = 8, learning_rate = 0.1, random_state = 101)

Model.fit(X, Y)

YPredict = Model.predict(TestSet)

Submission.insert(loc = 1, column = 'item_cnt_month', value = YPredict)

Submission.to_csv('Sub1.csv', index = False)